In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

In [2]:
import pandas as pd
from datetime import datetime


pos = 0
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())




,login,senha,ubs
0,joana,05216791418,borges


In [3]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [4]:
def clear_race_field():
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(Keys.BACKSPACE)
    time.sleep(1)
    race.send_keys(Keys.BACKSPACE)
    time.sleep(1)
    race.click()

    pass

In [5]:



def people(race_item):
    
    openEpHealph()
    print(f'Extração iniciada as: {datetime.now()}')
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")
    
    nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
    time.sleep(1)
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    
    race.send_keys(Keys.ENTER)
    time.sleep(1)
    
    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()
    
    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [6]:
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_table(race_item):
    
    table = []
    people(race_item)
    
    exit = False
    
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    col = peoples.find_elements(By.TAG_NAME, 'td')
    
    while exit is False:
        try:
            for i in col:
                table.append(i.text)
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                print(ElementClickInterceptedException)
                
            if NoSuchElementException: 
                print(NoSuchElementException)
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            col = peoples.find_elements(By.TAG_NAME, 'td')
    
    nav.quit()
    return table    


In [7]:
def build_df(table, race_item):
    '''name cns cpf age gender last_visit profissional team unity'''

    df_people = pd.DataFrame()

    name = []
    cns = []
    cpf = []
    age = []
    gender = []
    last_visit = []
    profissional = []
    team = []
    unity = []
    race_col = []


    for nam in range(1, len(table), 10):
        name.append(table[nam])

    for cn in range(2, len(table), 10):
        cns.append(table[cn])

    for cp in range(3, len(table), 10):
        cpf.append(table[cp])

    for ag in range(4, len(table), 10):
        age.append(table[ag])

    for gen in range(5, len(table), 10):
        gender.append(table[gen])

    for lv in range(6, len(table), 10):
        last_visit.append(table[lv])

    for prof in range(7, len(table), 10):
        profissional.append(table[prof])

    for t in range(8, len(table), 10):
        team.append(table[t])

    for unt in range(9, len(table), 10):
        unity.append(table[unt])
    if table[0] != 'Nenhum indivíduo encontrado':
        for race in range(0, len(table), 10):
            race_col.append(raceList[race_item])

    df_people['name'] = name
    df_people['cns'] = cns
    df_people['cpf'] = cpf
    df_people['age'] = age
    df_people['age'] = df_people['age'].astype(str).str[4:-1]

    df_people['gender'] = gender
    df_people['last_visit'] = last_visit
    df_people['profissional'] = profissional
    df_people['team'] = team
    df_people['unity'] = unity
    df_people['race_col'] = race_col

#     display(df_people)
    
#     df_people.to_excel('tabela.xlsx', index=False)
    
    return df_people


In [8]:
def complet_extraction():
    
    i = 0
    complete_df = pd.DataFrame()
    pre_df = pd.DataFrame()
    pre_list = []
    
    for i in range(0, len(raceList)):
        pre_list = get_table(i)
        pre_df = build_df(pre_list, i)

        complete_df = pd.concat([complete_df, pre_df], ignore_index= True)
    
    display(complete_df)
    print(f'Extração finalizada as: {datetime.now()}')
    return complete_df
#         complete_df = pd.concat([complete_df, suap_df], ignore_index= True)

exibi = complet_extraction()


Extração iniciada as: 2024-01-20 11:43:17.025475
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
Extração iniciada as: 2024-01-20 11:43:41.993594
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
Extração iniciada as: 2024-01-20 11:44:06.607690
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
Extração iniciada as: 2024-01-20 11:44:31.309044
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
Extração iniciada as: 2024-01-20 11:44:56.227729
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
Extração iniciada as: 2024-01-20 11:45:20.909191
<class 'selenium.common.exceptions.ElementClickIntercept

,name,cns,cpf,age,gender,last_visit,profissional,team,unity,race_col
0,AMÉLIA JULIETE DE FREITAS BRITO,700005032962907,09928726426,17/09/1992,Feminino,29/11/2023,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
1,ANA PAULA PEREIRA LEITE,700105917353518,11575142430,14/05/1996,Feminino,06/12/2023,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
2,CAROLINY GOMES DE OLIVEIRA PEREIRA,706100880344930,05574364433,16/08/1999,Feminino,18/01/2024,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
3,HELLEN AZEVEDO MARINHO,898000604249278,06826745402,24/07/1986,Feminino,28/11/2023,ALYSSON TIAGO MARQUES BRANDAO (CNS 70860501793...,0004,PSF BORGES,BRANCA
4,LYZETE VERAS DE BRITO NOBRE,704002897979760,11386245437,25/10/1999,Feminino,18/01/2024,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
5,MARIA JACINTA DA SILVA ALMEIDA,707007822544235,,09/09/1993,Feminino,13/12/2023,PAULA SUZILANEA NICACIO VERAS (CNS 70890378083...,0004,PSF BORGES,BRANCA
6,ANA ALICE DA SILVA ALMEIDA,702401548215721,06396454459,28/08/1986,Feminino,11/10/2023,ALYSSON TIAGO MARQUES BRANDAO (CNS 70860501793...,0004,PSF BORGES,PARDA
7,ANA ISABEL MORGANA DOS ANJOS SERAFIM,708209106611441,06190311407,08/09/1987,Feminino,08/01/2024,DANIELLA KELLY LUCAS DE MORAIS RAMOS (CNS 7002...,0004,PSF BORGES,PARDA
8,ANDREZZA SAMARA DO NASCIMENTO PAZ,705406493958395,09574195465,09/03/1993,Feminino,15/01/2024,ALYSSON TIAGO MARQUES BRANDAO (CNS 70860501793...,0004,PSF BORGES,PARDA
9,ERIVONEIDE BEZERRA DO AMARAL,705001065919951,09009859410,18/02/1989,Feminino,05/12/2023,PAULA SUZILANEA NICACIO VERAS (CNS 70890378083...,0004,PSF BORGES,PARDA


Extração finalizada as: 2024-01-20 11:45:39.933499


In [12]:
print(type(exibi))
print(exibi.info())
display(exibi)
exibi.to_excel(f'pregnant/{data_e_hora}_{autentica["ubs"][0]}_pregnant.xlsx', index=False)
# people(2)

# clear_race_field()
# jeba = get_table(0)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          22 non-null     object
 1   cns           22 non-null     object
 2   cpf           22 non-null     object
 3   age           22 non-null     object
 4   gender        22 non-null     object
 5   last_visit    22 non-null     object
 6   profissional  22 non-null     object
 7   team          22 non-null     object
 8   unity         22 non-null     object
 9   race_col      22 non-null     object
dtypes: object(10)
memory usage: 1.8+ KB
None


,name,cns,cpf,age,gender,last_visit,profissional,team,unity,race_col
0,AMÉLIA JULIETE DE FREITAS BRITO,700005032962907,09928726426,17/09/1992,Feminino,29/11/2023,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
1,ANA PAULA PEREIRA LEITE,700105917353518,11575142430,14/05/1996,Feminino,06/12/2023,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
2,CAROLINY GOMES DE OLIVEIRA PEREIRA,706100880344930,05574364433,16/08/1999,Feminino,18/01/2024,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
3,HELLEN AZEVEDO MARINHO,898000604249278,06826745402,24/07/1986,Feminino,28/11/2023,ALYSSON TIAGO MARQUES BRANDAO (CNS 70860501793...,0004,PSF BORGES,BRANCA
4,LYZETE VERAS DE BRITO NOBRE,704002897979760,11386245437,25/10/1999,Feminino,18/01/2024,IVANEIDE OLIVEIRA DOS SANTOS (CNS 708800709049...,0004,PSF BORGES,BRANCA
5,MARIA JACINTA DA SILVA ALMEIDA,707007822544235,,09/09/1993,Feminino,13/12/2023,PAULA SUZILANEA NICACIO VERAS (CNS 70890378083...,0004,PSF BORGES,BRANCA
6,ANA ALICE DA SILVA ALMEIDA,702401548215721,06396454459,28/08/1986,Feminino,11/10/2023,ALYSSON TIAGO MARQUES BRANDAO (CNS 70860501793...,0004,PSF BORGES,PARDA
7,ANA ISABEL MORGANA DOS ANJOS SERAFIM,708209106611441,06190311407,08/09/1987,Feminino,08/01/2024,DANIELLA KELLY LUCAS DE MORAIS RAMOS (CNS 7002...,0004,PSF BORGES,PARDA
8,ANDREZZA SAMARA DO NASCIMENTO PAZ,705406493958395,09574195465,09/03/1993,Feminino,15/01/2024,ALYSSON TIAGO MARQUES BRANDAO (CNS 70860501793...,0004,PSF BORGES,PARDA
9,ERIVONEIDE BEZERRA DO AMARAL,705001065919951,09009859410,18/02/1989,Feminino,05/12/2023,PAULA SUZILANEA NICACIO VERAS (CNS 70890378083...,0004,PSF BORGES,PARDA


In [10]:
# build_df(jeba, 0)